In [1]:
import seaborn as sns 
from transformers import AdamW, T5ForConditionalGeneration, T5TokenizerFast as T5Tokenizer
import pytorch_lightning as pl
import torch
from torch.utils.data import Dataset, DataLoader
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger
import textwrap
from pathlib import Path

c:\Users\hp\OneDrive\Desktop\ENV\env\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\hp\OneDrive\Desktop\ENV\env\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: Could not find module 'C:\Users\hp\OneDrive\Desktop\ENV\env\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warn(f"Failed to load image Python extension: {e}")


In [21]:
import gc

In [29]:
if torch.cuda.is_available():  
  dev = "cuda:0" 
else:  
  dev = "cpu" 
device = torch.device(dev)

In [2]:
import seaborn as sns
from tqdm.auto import tqdm
from pylab import rcParams
import matplotlib.pyplot as plt 
from matplotlib import rc 
import pandas as pd
from sklearn.model_selection import train_test_split

In [3]:
from datasets import load_dataset

In [4]:
dataset = load_dataset("samsum")

Found cached dataset samsum (C:/Users/hp/.cache/huggingface/datasets/samsum/samsum/0.0.0/f1d7c6b7353e6de335d444e424dc002ef70d1277109031327bc9cc6af5d3d46e)
100%|██████████| 3/3 [00:00<00:00, 476.79it/s]


In [5]:
pl.seed_everything(42)

Global seed set to 42


42

In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [7]:
from torch.utils.data import DataLoader

dataset.set_format(type="torch", columns=["id","dialogue", "summary"])
dataloader = DataLoader(dataset, batch_size=4)

In [8]:
dataset["train"].shape

(14732, 3)

In [9]:
dataset["test"].shape

(819, 3)

In [10]:
dataset["validation"].shape

(818, 3)

In [11]:
dataset["validation"]

Dataset({
    features: ['id', 'dialogue', 'summary'],
    num_rows: 818
})

In [12]:
train_df=pd.DataFrame(dataset['train'])

In [13]:
train_df

,id,dialogue,summary
0,13818513,Amanda: I baked cookies. Do you want some?\r\...,Amanda baked cookies and will bring Jerry some...
1,13728867,Olivia: Who are you voting for in this electio...,Olivia and Olivier are voting for liberals in ...
2,13681000,"Tim: Hi, what's up?\r\nKim: Bad mood tbh, I wa...",Kim may try the pomodoro technique recommended...
3,13730747,"Edward: Rachel, I think I'm in ove with Bella....",Edward thinks he is in love with Bella. Rachel...
4,13728094,Sam: hey overheard rick say something\r\nSam:...,"Sam is confused, because he overheard Rick com..."
...,...,...,...
14727,13863028,Romeo: You are on my ‘People you may know’ lis...,Romeo is trying to get Greta to add him to her...
14728,13828570,Theresa: <file_photo>\r\nTheresa: <file_photo>...,Theresa is at work. She gets free food and fre...
14729,13819050,John: Every day some bad news. Japan will hunt...,Japan is going to hunt whales again. Island an...
14730,13828395,Jennifer: Dear Celia! How are you doing?\r\nJe...,Celia couldn't make it to the afternoon with t...


In [14]:
test_df=pd.DataFrame(dataset['test'])

In [15]:
val_df=pd.DataFrame(dataset['validation'])

In [16]:
from simplet5 import SimpleT5

model = SimpleT5()
model.from_pretrained(model_type="t5", model_name="t5-base")

Global seed set to 42
Downloading: 100%|██████████| 773k/773k [00:01<00:00, 467kB/s]  
Downloading: 100%|██████████| 1.32M/1.32M [00:02<00:00, 545kB/s] 
Downloading: 100%|██████████| 1.17k/1.17k [00:00<00:00, 625kB/s]
Downloading: 100%|██████████| 850M/850M [06:59<00:00, 2.13MB/s] 


In [19]:
train_df['dialogue'] = "summarize: " + train_df['dialogue']
train_df
test_df['dialogue'] = "summarize: " + test_df['dialogue']
test_df
val_df['dialogue'] = "summarize: " + val_df['dialogue']
val_df

,id,dialogue,summary
0,13817023,"summarize: summarize: A: Hi Tom, are you busy ...",A will go to the animal shelter tomorrow to ge...
1,13716628,summarize: summarize: Emma: I’ve just fallen i...,Emma and Rob love the advent calendar. Lauren ...
2,13829420,summarize: summarize: Jackie: Madison is pregn...,Madison is pregnant but she doesn't want to ta...
3,13819648,summarize: summarize: Marla: <file_photo>\r\nM...,Marla found a pair of boxers under her bed.
4,13728448,summarize: summarize: Robert: Hey give me the ...,Robert wants Fred to send him the address of t...
...,...,...,...
813,13829423,summarize: summarize: Carla: I've got it...\r\...,Carla's date for graduation is on June 4th. Di...
814,13727710,"summarize: summarize: Gita: Hello, this is Bet...",Bev is going on the school trip with her son. ...
815,13829261,summarize: summarize: Julia: Greg just texted ...,Greg cheated on Julia. He apologises to her. R...
816,13680226,summarize: summarize: Marry: I broke my nail ;...,Marry broke her nail and has a party tomorrow....


In [41]:
model.train(train_df=train_df,
            eval_df=val_df, 
            source_max_token_len=512, 
            target_max_token_len=128, 
            batch_size=8, max_epochs=5)

MisconfigurationException: `CUDAAccelerator` can not run on your system since the accelerator is not available. The following accelerator(s) is available and can be passed into `accelerator` argument of `Trainer`: ['cpu'].

In [ ]:
! ( cd outputs; ls )

The system cannot find the path specified.
